In [1]:
import pandas as pd
import numpy as np
import os
import warnings
import time
import re

import json

from skrub import TableReport
from ollama import Client
from pdfquery import PDFQuery
from pprint import pprint
from IPython.display import display, HTML

import bid_utils

In [2]:
## load the dfs built in 1_collect_files
rep_drive_ATAE =  r"C:\Users\jch_m\ATAE"
rep_data_input =  r"C:\Users\jch_m\ATAE\Nicolas MONCEAU - NICOLAS\Stage_AppelsOffres\data_input"
rep_data_output = r"C:\Users\jch_m\ATAE\Nicolas MONCEAU - NICOLAS\Stage_AppelsOffres\data_output"

path_df_EBP = os.path.join(rep_data_output, "df_EBP.pkl")
df_EBP = pd.read_pickle(path_df_EBP)

path_df_consult = os.path.join(rep_data_output, "df_consult.pkl")
df_consult = pd.read_pickle(path_df_consult)

path_df_rejet = os.path.join(rep_data_output, "df_rejet.pkl")
df_rejet = pd.read_pickle(path_df_rejet)


In [3]:
# Merge df_consult and df_EBP pour sync EBP ID on the mission/files
# note: no need to sync EBP with rejet, because none of the Rejet file match an EBP entry (dans la liste des repertoires et fichiers). normal ?

# Define the columns to match on
match_columns = ['SPS Name', 'Ville', 'Entreprise', 'Mission']

df_consult_ebp = pd.merge(df_consult, df_EBP, on=match_columns, how='left', suffixes=('_consult', '_ebp'))
df_consult_ebp['ID EBP'] = df_consult_ebp['ID EBP_ebp'].fillna("no EBP")
df_consult_ebp = df_consult_ebp.drop(columns=['ID EBP_ebp','ID EBP_consult', 'statut_ebp'])

#df_consult_ebp['file_name'] = df_consult_ebp['file_path'].str.split(r'\\').str[-1].strip()
df_consult_ebp['file_name'] = df_consult_ebp['file_path'].str.split(r'\\').str[-1].str.strip()

In [4]:
# define a unique 'no EBP xx'  for each combinaison of SPS+Ville+Entreprise+Mission, for all related files
#
mask = df_consult_ebp["ID EBP"] == "no EBP"

# Create the combined series only for the masked rows
combined_series_for_update = (
    df_consult_ebp.loc[mask, 'SPS Name'].astype(str) + '_' +
    df_consult_ebp.loc[mask, 'Ville'].astype(str) + '_' +
    df_consult_ebp.loc[mask, 'Entreprise'].astype(str) + '_' +
    df_consult_ebp.loc[mask, 'Mission'].astype(str)
)

# Generate unique IDs for this series
unique_ids, _ = pd.factorize(combined_series_for_update)

# Assign back to the original DataFrame using the mask
df_consult_ebp.loc[mask, "ID EBP"] = "no EBP " + (unique_ids + 1).astype(str)


In [5]:
# Add columns "AO_docs" (True/False), "AO_Doc_type"(CCTP, CCAP, RC, AAPC, Memo_tech), "Commande"(True/False)

# Set AO_doc_type
# Initialize all with default value
df_consult_ebp['AO_doc_type'] = 'no type'

# Define mask for inclusion et exclusion 
df_consult_ebp['AO_docs'] = (
    (df_consult_ebp['file_path'].str.lower().str.contains('devis') & df_consult_ebp['file_path'].str.lower().str.contains(r'\\consul')) | 
    (df_consult_ebp['file_path'].str.lower().str.contains(r'\\adm') & df_consult_ebp['file_path'].str.lower().str.contains(r'\\consul'))
)
mask = df_consult_ebp['AO_docs'] == True

# Création d'un masque d'exclusion : lot est retiré provisoirement
mots_cles_a_exclure = ["plan ", "assurance", "honneur", "plans", "coupe", "vue", "facade", "archi"] 
exclusion_mask = df_consult_ebp.loc[mask, 'file_path'].str.lower().apply(
    lambda x: any(mot in x for mot in mots_cles_a_exclure)
)

# Sélection des lignes à inclure (qui ne contiennent pas les mots-clés à exclure)
inclusion_mask = mask.copy()
inclusion_mask.loc[mask] = ~exclusion_mask

# par ordre inverse d'importance, pour que les derniers checks ecrasent eventuellement les premiers
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('sps', case=False, na=False), 'AO_doc_type'] = 'CCTP'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('PA-', case=False, na=False), 'AO_doc_type'] = 'Procedure Adaptee'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('RC', case=False, na=False), 'AO_doc_type'] = 'Reglement'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('lettre', case=False, na=False), 'AO_doc_type'] = 'Lettre Consult'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('courier', case=False, na=False), 'AO_doc_type'] = 'Lettre Consult'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('consult', case=False, na=False), 'AO_doc_type'] = 'Lettre Consult'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('AAPC', case=False, na=False), 'AO_doc_type'] = 'AAPC' #Achat avec concurence
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('glement', case=False, na=False), 'AO_doc_type'] = 'Reglement'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('planning', case=False, na=False), 'AO_doc_type'] = 'Planning'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('phasag', case=False, na=False), 'AO_doc_type'] = 'Planning'
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('programme', case=False, na=False), 'AO_doc_type'] = 'CCTP' # Technique
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('CCTP', case=False, na=False),'AO_doc_type'] = 'CCTP' # Technique
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('CCP', case=False, na=False), 'AO_doc_type'] = 'CCP' # General , Technique et Admin
df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('CCAP', case=False, na=False), 'AO_doc_type'] = 'CCAP' #Administratif
#df_consult_ebp.loc[inclusion_mask & df_consult_ebp['file_name'].str.contains('DCE', case=False, na=False), 'AO_doc_type'] = 'CCTP' # Dossier Consult 



In [6]:
print("nb total de fichiers:",len(df_consult_ebp['AO_doc_type']))
print("nb total de fichiers de consultation:",len(df_consult_ebp[df_consult_ebp['AO_docs']==True] ))
print("nb de fichiers AO_docs avec un type identifié:",len(df_consult_ebp['AO_doc_type']) - len(df_consult_ebp[df_consult_ebp['AO_doc_type']=='no type']))
print("\nDétails par type", df_consult_ebp['AO_doc_type'].value_counts())


nb total de fichiers: 156914
nb total de fichiers de consultation: 3103
nb de fichiers AO_docs avec un type identifié: 830

Détails par type AO_doc_type
no type              156084
CCTP                    348
Reglement               243
CCAP                    180
AAPC                     28
Lettre Consult           12
Planning                  9
Procedure Adaptee         7
CCP                       3
Name: count, dtype: int64


In [7]:
# Liste de fichiers de consultations sans type clairement identifié (ou volontairement exclus)
#
list_file_AO_notype = ""
mask2 = (df_consult_ebp['AO_docs'] == True) &  (df_consult_ebp['AO_doc_type'] == 'no type')
list_file_AO_notype = [df_consult_ebp.loc[mask2, 'file_name'], df_consult_ebp.loc[mask2, 'file_path']]
for index, row in df_consult_ebp.loc[mask2, ['file_name', 'file_path']].iterrows():
    file = row['file_name']
    file_path = row['file_path'].strip()

    if any(mot in file.lower() for mot in mots_cles_a_exclure):
        pass

    bid_utils.path_to_link(file_path)


In [8]:
TableReport(df_consult_ebp)

Processing column  10 / 10


,,,,,,,,,,


In [9]:
# Creation d'un DF_consult_elevated avec une ligne par "EBP ID", 
# et l'ajout des features resultants de l'extraction 

df_consult_ao = df_consult_ebp[df_consult_ebp['AO_docs']==True]
df_consult_elevated = df_consult_ao.drop_duplicates(subset=["ID EBP"], keep="first")

# Suppression des colonnes desormais inutiles
df_consult_elevated = df_consult_elevated.drop(columns=['file_path','file_name','AO_docs','AO_doc_type'])

"""
df_consult_elevated['lieu'] = ""
df_consult_elevated['type travaux'] = ""
df_consult_elevated['duree travaux'] = 0
df_consult_elevated['prix travaux'] = 0.0
df_consult_elevated['maitre ouvrage'] = ""
df_consult_elevated['maitre oeuvre'] = ""
"""

'\ndf_consult_elevated[\'lieu\'] = ""\ndf_consult_elevated[\'type travaux\'] = ""\ndf_consult_elevated[\'duree travaux\'] = 0\ndf_consult_elevated[\'prix travaux\'] = 0.0\ndf_consult_elevated[\'maitre ouvrage\'] = ""\ndf_consult_elevated[\'maitre oeuvre\'] = ""\n'

In [10]:
df_consult_elevated.info()


<class 'pandas.core.frame.DataFrame'>
Index: 926 entries, 0 to 156769
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   SPS Name        926 non-null    object
 1   Ville           926 non-null    object
 2   Entreprise      926 non-null    object
 3   Mission         926 non-null    object
 4   statut_consult  926 non-null    object
 5   ID EBP          926 non-null    object
dtypes: object(6)
memory usage: 50.6+ KB


# Test de modeles pour resumer et extraire l'info un CCxP 

In [11]:
#prompt_role = "tu es un assistant pour analyser les appels d'offres de coordination SPS, extraire les informations demandees en suivant les instructions et le format attendu"
prompt_role = "your are an assistant to analyse the bids for Coordination SPS, extract key informations followins the specified instructions in term of format and content"

prompt_task_resumer = "Extrait du texte les elements suivants : \n\
'Nom Chantier': scope du projet, objet du chantier, objectif du programme ;\n\
'Lieu du Chantier': ville, Commune, Departement, Rue ; \n\
'Maitre ouvrage': nom du Maitre d'ouvrage du projet ;\n\
'Maitre oeuvre': nom du maitre d'oeuvre du chantier  ;\n\
'Type de travaux': type et nature du travaux du chantier exemple: amenagement, construction ;\n \
'Planning previsionnel': dates du chantier par lot, par phase ; calendrier operationel ;\n\
'Prix des travaux (en euros)': exemple <10_000> ;\n\
'Duree Previsionnelle des Travaux (en mois)': exemple <8> ;\n\
'Categorie operation SPS':  I, II, ou III ;\n\
Réponds en respectant la structure ci-dessus \n\
Donne uniquement les informations disponibles dans le texte , sans interpretation ou estimation. \n\
Donne des reponses chiffrées et quantifiées lorsque c'est possible. \n\
Ne mentionne pas **Autres informations:**  \n\
La reponse doit etre ecrite en francais et avoir un maximum 900 mots \n \
Texte: "

prompt_task_json = "En Francais, Formate les informations extraites et dans le texte, avec une structure JSON, en respectant strictement les clefs et types specifies tels que \n \
{ \
\"Nom Chantier\": \"<string>\", \
\"Lieu du Chantier\": \"<string>\",\
\"Maitre ouvrage\": \"<string>\", \
\"Maitre oeuvre\": \"<string>\", \
\"Type de Travaux\": \"<string>\", \
\"Planning previsionnel\": \"<string>\", \
\"Prix des travaux (en euros)\": \"<integer>\" , \
\"Durée Prévisionnelle des Travaux (en mois)\": \"<integer>\" ,\
} \n\
Donne des reponses uniquement pour ces clefs. en chiffre lorsque c'est possible. Si pas d'info, laisser vide  \n "

prompt_task_json_uniq = "En Francais, Formate les informations extraites et dans le texte, avec une structure JSON, en respectant strictement les clefs et types specifies tels que \n \
{ \
  \"Nom Chantier\": \"<string>\" scope du projet, objet du chantier, objectif du programme, \
  \"Lieu du Chantier\": \"<string>\"  ville  Commune  Departement Rue ,\
  \"Maitre ouvrage\": \"<string>\" nom du Maitre d'ouvrage du projet,\
  \"Maitre oeuvre\": \"<string>\" nom du maitre d'oeuvre du chantier ,\
  \"Type de Travaux\": \"<string>\" type et nature du travaux du chantier exemple: amenagement construction ,\
  \"Planning phase conception\": \"<string>\" date et duree Previsionnelle , \
  \"Planning phase realisation\": \"<string>\" date et duree Previsionnelle ,\
  \"Prix des travaux (en euros)\": \"<integer>\" ,\
  \"Durée Prévisionnelle des Travaux (en mois)\": \"<integer>\" ,\
  \"Categorie operation SPS\": \"<string>\" I II ou III,\
} \n\
Réponds en respectant la structure ci-dessus \n\
Donne uniquement les informations disponibles dans le texte , sans interpretation ou estimation. \n\
Donne des reponses chiffrées et quantifiées lorsque c'est possible. \n\
La reponse doit etre ecrite en francais et avoir un maximum 900 mots \n \
Texte: \n "
format_json_0 = {
    "Nom Chantier": "<string>",
    "Lieu du Chantier": "<string>",
    "Maitre ouvrage": "<string>",
    "Maitre oeuvre": "<string>",
    "Type de Travaux": "<string>",
    "Planning phase conception": "<string>",
    "Planning phase realisation": "<string>",
    "Prix des travaux": "<integer>",
    "Duree des travaux": "<integer>",
    "Categorie operation SPS": "<string>",
} 

format_json = {
    "Nom Chantier": "",
    "Lieu du Chantier": "",
    "Maitre ouvrage": "",
    "Maitre oeuvre": "",
    "Type de Travaux": "",
    "Planning phase conception": "",
    "Planning phase realisation": "",
    "Prix des travaux": 0,
    "Duree des travaux": 0,
    "Categorie operation SPS": "",
} 
json_string = json.dumps(format_json, ensure_ascii=False, indent=2)

prompt_task_resumer2 = f"En Francais, extrairr du **Texte CCxP** les **Informations** suivantes : \n  \
**Informations**:\n\
Nom Chantier: scope du projet, objet du chantier, objectif du programme, \n\
Lieu du Chantier: ville  Commune  Departement Rue , \n\
Maitre ouvrage: nom du Maitre d'ouvrage du projet,\n\
Maitre oeuvre: nom du maitre d'oeuvre du chantier ,\n\
Type de Travaux: type et nature du travaux du chantier exemple: amenagement construction ,\n\
Planning phase conception: date et duree Previsionnelle, \n\
Planning phase realisation: date et duree Previsionnelle, \n\
Prix des travaux : en euros , type integer \n\
Durée Prévisionnelle des Travaux: : en nombre de mois, type integer \n\
Categorie operation SPS:  I II ou III, \n\n\
**Instructions**: \n\
Donne uniquement les informations disponibles dans le texte CCxP, sans interpretation ou estimation. \n\
Donne des reponses chiffrées et quantifiées lorsque c'est possible. \n\
La reponse doit etre ecrite en francais et avoir un maximum 900 mots \n \
Réponds uniquement aux informations demandées qui sont toutes dans le texte\n\
**Texte CCxP**: \n "

prompt_task_json_uniq2 = f"En Francais, extraire du **Texte CCxP** les **Informations** suivantes : \n  \
**Informations**:\n\
Nom Chantier: scope du projet, objet du chantier, objectif du programme, \n\
Lieu du Chantier: ville  Commune  Departement Rue , \n\
Maitre ouvrage: nom du Maitre d'ouvrage du projet,\n\
Maitre oeuvre: nom du maitre d'oeuvre du chantier ,\n\
Type de Travaux: type et nature du travaux du chantier exemple: amenagement construction ,\n\
Planning phase conception: date et duree Previsionnelle, \n\
Planning phase realisation: date et duree Previsionnelle, \n\
Prix des travaux : en euros , type integer \n\
Durée Prévisionnelle des Travaux: : en nombre de mois, type integer \n\
Categorie operation SPS: categorie ou type  I II ou III ou 1,2, 3, \n\n\
**Instructions**: \n\
Donne uniquement les informations disponibles dans le texte , sans interpretation ou estimation. \n\
Donne des reponses chiffrées et quantifiées lorsque c'est possible. \n\
La reponse doit etre ecrite en francais et avoir un maximum 900 mots \n\n \
Réponds uniquement avec la structure JSON ci-dessus : \n{json_string}\n\n\
**Texte CCxP**: \n "

prompt_task_json_uniq3 = f"Format the output, focusing on informations requested and available within **Texte CCxP** and initial document . Do not invent anything. \
Strickly follow the JSON format request. Write in French: \n{json_string}\n\n \
**Texte CCxP**: \n "

pr_resum_0 = f"In French, answer the following **Questions** based on data provided with **Texte CCxP**  : \n  \
**Questions**:\n\
Quel est le nom du Chantier? scope du projet, objet du chantier, objectif du programme, \n\
Quel est le Lieu du Chantier ? ville  Commune  Departement Rue , \n\
Qui est le Maitre ouvrage ? nom du Maitre d'ouvrage du batiment,\n\
Qui est le Maitre oeuvre ? nom du maitre d'oeuvre du chantier \n \
**Instructions**: \n\
Answer only in french. Answer to the specified questions based on available data from **Texte CCxP** \n\
Give quantified values (in euros, in months) when possible. \n\
The answer should be less than 900 words \n \
**Texte CCxP**: \n "

pr_resum_1 = f"In French, answer the following **Questions** based on data provided with **Texte CCxP**  : \n  \
**Questions**:\n\
Quel est le nom du Chantier? scope du projet, objet du chantier, objectif du programme \n\
Quel est le Lieu du Chantier ? ville  Commune  Departement Rue  \n\
Qui est le Maitre Ouvrage ? nom du Maitre d'Ouvrage du batiment\n\
Qui est le Maitre Oeuvre ? nom du Maitre d'Oeuvre du chantier \n\
Type de Travaux sont prevus ?: type et nature du travaux du chantier exemple: amenagement construction ,\n\
Combien de lot sont prevus ? nombre et type de lot ? \n \
Quel est le Planning phase conception ? date et duree Previsionnelle, \n\
Quel est le Planning phase realisation ? date et duree Previsionnelle, \n\
Quel est le Prix des travaux ?: en euros \n\
Quelle est la Durée Prévisionnelle des Travaux ? en nombre de mois, type integer \n\
Quelle est la Categorie operation SPS ? categorie I, II ou III, \n\n\
**Instructions**: \n\
Answer only in french. Answer to the specified questions based on available data from **Texte CCxP** \n\
Give quantified values (in euros, in months) when possible. \n\
The answer should be less than 900 words \n \
**Texte CCxP**: \n "

pr_resum_2 = f"En Francais, extraire du texte fourni Informations** suivantes : \n  \
**Informations**:\n\
Nom Chantier: scope du projet, objet du chantier, objectif du programme, \n\
Lieu du Chantier: ville  Commune  Departement Rue , \n\
Maitre ouvrage: nom du Maitre d'ouvrage du projet,\n\
Maitre oeuvre: nom du maitre d'oeuvre du chantier ,\n\
Type de Travaux: type et nature du travaux du chantier exemple: amenagement construction ,\n\
Planning phase conception: date et duree Previsionnelle, \n\
Planning phase realisation: date et duree Previsionnelle, \n\
Prix des travaux : en euros , type integer \n\
Durée Prévisionnelle des Travaux: : en nombre de mois, type integer \n\
Categorie operation SPS: categorie ou type  I II ou III ou 1,2, 3, \n\n\
**Instructions**: \n\
Donne uniquement les informations disponibles dans le texte , sans interpretation ou estimation. \n\
Donne des reponses chiffrées et quantifiées lorsque c'est possible. \n\
La reponse doit etre ecrite en francais et avoir un maximum 900 mots \n\n \
Réponds uniquement avec la structure JSON ci-dessus : \n{json_string}\n"


pr_intro_1 = "Please read carrefully the document provided in the next message, Focus on the key questions specified and provide answer"


In [12]:
# Prompts pour la collection des infos par type de document
#
prompt_role = "your are an assistant to analyse the bids for Coordination SPS, extract key informations followins the specified instructions in term of format and content"

###### ---- CCTP -------------- ###################
format_json_cctp = {
    "Nom Chantier": "",
    "Lieu du Chantier": "",
    "Maitre ouvrage": "",
    "Maitre oeuvre": "",
    "Type de Travaux": "",
    "Planning phase conception": "",
    "Planning phase realisation": "",
    "Prix des travaux": 0,
    "Duree des travaux": 0,
    "Categorie operation SPS": "",
} 
json_string_cctp = json.dumps(format_json_cctp, ensure_ascii=False, indent=2)

prompt_resumer_cctp = """
En Francais, extraire du **Texte CCxP** les **Informations** suivantes : 

**Informations**:
'Nom Chantier': scope du projet, objet du chantier, objectif du programme, 
'Lieu du Chantier': ville  Commune  Departement Rue , 
'Maitre ouvrage': nom du Maitre d'ouvrage du projet,
'Maitre oeuvre': nom du maitre d'oeuvre du chantier ,
'Type de Travaux': type et nature du travaux du chantier exemple: amenagement construction ,
'Planning phase conception': date et duree Previsionnelle, 
'Planning phase realisation': date et duree Previsionnelle, 
'Prix des travaux' : en euros HT <integer>, 
'Durée Prévisionnelle des Travaux': en nombre de mois <integer>,
'Categorie operation SPS':  I II ou III, 

**Instructions**: 
Donne uniquement les informations disponibles dans le texte CCxP, sans interpretation ou estimation. 
Donne des reponses chiffrées et quantifiées lorsque c'est possible. 
La reponse doit etre ecrite en francais et avoir un maximum 900 mots 
Réponds uniquement aux informations demandées qui sont toutes dans le texte

**Texte CCxP**: 
"""

prompt_json_cctp = f"""
Format the output, focusing on informations requested and available within the **Text** and initial document . 
Do not invent anything. 
Strickly follow the JSON format request. Write in French: \n{json_string_cctp}\

**Text**: 
"""

###### ---- Reglement -------------- ###################
format_json_regl = {
    "Critere Prix": 0,
    "Critere Technique": {
        "Moyen Humain et Experience" : 0,
        "Methodologie" : 0,
        "Cohérence du temps" : 0,
        "Compréhension des enjeux" : 0,
    },
    "Prix des travaux": 0 ,
    "Duree des travaux": 0,
} 
string_json_regl = json.dumps(format_json_regl, ensure_ascii=False, indent=2)

prompt_resumer_regl = """ 
Extract from **Texte** the following informations related to 'evaluation et critère de l'offre'  : 

**Informations**:
'Critère Prix': quel poids ou ponderation pour le critère de l'examen du prix des prestations , valeur de la note maximale ? <integer>,
'Critère Technique' : quels poids ou ponderations pour le critère de l'examen la valeur technique de l'offre ? et detaillant si disponible les sous-critères suivants <integer>,
'Moyen Humain et Experience': quels poids pour la compétences, moyen matériel ? <integer>,
'Methodologie': quels poids pour ce sous-critère la methode technique ? <integer>,
'Cohérence du temps': quels poids pour le temps de travail estimé pour ce service ? <integer>,
'Compréhension des enjeux': , 
'Prix des travaux' : en euros HT <integer>, 
'Durée Prévisionnelle des Travaux': en nombre de mois <integer>,

**Instructions**: 
Donne uniquement les informations disponibles dans le texte , sans interpretation ou estimation. 
Donne des reponses chiffrées et quantifiées lorsque c'est possible. 
La reponse doit etre ecrite en francais et avoir un maximum 500 mots 
Réponds uniquement aux informations demandées qui sont toutes dans le texte

**Texte**: 
"""

prompt_json_regl = f"""
Format the output, focusing on informations requested and available within the **Text** and initial document . 
Do not invent anything. 
Strickly follow the JSON format request. Write in French: 
{string_json_regl}

**Text**: \n 
""" 

###### ---- AAPC -------------- ###################
format_json_aapc = {
    "Mission": "",
    "Lieu du Chantier": "",
    "Maitre ouvrage": "",
    "Lot": "",
    "Tranche": "",
    "Prix des travaux": 0,
    "Duree des travaux": 0,
}
string_json_aapc = json.dumps(format_json_aapc, ensure_ascii=False, indent=2)

prompt_resumer_aapc = """ 
Extract from **Texte** the following informations related to 'evaluation et critère de l'offre'  : 

**Informations**:
'Mission': scope du projet, objet du chantier, objectif du programme, 
'Lieu du Chantier': ville  Commune  Departement Rue , 
'Maitre ouvrage': nom du Maitre d'ouvrage du projet,
'Tranche': le projet est il decoupé en plusieurs Tranche ou phases ex '2 tranches', 'non'
'Lot': le projet est il decoupé en plusieurs lots, ex '3 lots', 'non',
'Prix des travaux' : en euros HT, <integer>,
'Durée des Travaux': en nombre de mois, <integer>,

**Instructions**: 
Donne uniquement les informations disponibles dans le texte , sans interpretation ou estimation. 
Donne des reponses chiffrées et quantifiées lorsque c'est possible. 
La reponse doit etre ecrite en francais et avoir un maximum 500 mots 
Réponds uniquement aux informations demandées qui sont toutes dans le texte

**Texte**: 
"""

prompt_json_aapc = f"""
Format the output, focusing on informations requested and available within the **Text** and initial document . 
Do not invent anything. 
Strickly follow the JSON format request. Write in French: 
{string_json_aapc}

**Text**: \n 
""" 

###### ---- CCAP -------------- ###################
format_json_ccap= {
    "Objet du marché": "",
    "Lieu du Chantier": "",
    "Maitre ouvrage": "",
    "Maitre oeuvre": "",
    "tranche": "",
    "Lot": "",
}
string_json_ccap = json.dumps(format_json_ccap, ensure_ascii=False, indent=2)

prompt_resumer_ccap = """ 
Extract from **Texte** the following informations related to 'evaluation et critère de l'offre'  : 

**Informations**:
'Objet du marché': scope du projet, objet du chantier, objectif du programme, 
'Lieu du Chantier': ville  Commune  Departement Rue , 
'Maitre ouvrage': nom du Maitre d'ouvrage du projet,
'Maitre oeuvre': nom du Maitre d'oeuvre du projet, en phase de conception et de realisation 
'Tranche': le projet est il decoupé en plusieurs tranches, ex '2 tranches', 'non',
'Lot': le projet est il decoupé en plusieurs lot, ex '3 lots', 'non',

**Instructions**: 
Donne uniquement les informations disponibles dans le texte , sans interpretation ou estimation. 
Donne des reponses chiffrées et quantifiées lorsque c'est possible. 
La reponse doit etre ecrite en francais et avoir un maximum 500 mots 
Réponds uniquement aux informations demandées qui sont toutes dans le texte

**Texte**: 
"""

prompt_json_ccap = f"""
Format the output, focusing on informations requested and available within the **Text** and initial document . 
Do not invent anything. 
Strickly follow the JSON format request. Write in French: 
{string_json_ccap}

**Text**: \n 
""" 


In [13]:
# Init Ollama, with list of model, prompt

ollama_url = "http://localhost:11434"

# Get an ollama client
llmclient = Client(host=ollama_url)
model_options = {
    "num_predict": 1300,  # max number of tokens to predict
    "temperature": 0.05,
    "top_p": 0.2,
    "num_ctx": 9000 # max number of tokens to input
}
model_options_json = {
    "num_predict": 1300,  # max number of tokens to predict
    "temperature": 0.05,
    "top_p": 0.2,
    "format": "json",
    "num_ctx": 9000 # max number of tokens to input
}

# 'mistral-small3.1=14G  llama3.2:latest=2G gemma3:4b=3.3G
#list_model = ["gemma3:4b", "llama3.2", "minicpm-v", "mistral-small3.1"]
list_model = ["gemma3:12b", "gemma2:9b", "llama3.1:8b", "mistral:7b"]



In [14]:
## Run all models for the same task (prompt + file) for comparison
##

def run_model_CCTP(text_from_file, model_name):
    #Init system:
    llmclient.chat(model=model_name, options=model_options, messages=[{'role':'system','content': prompt_role}])
    
    #Resumer
    prompt_full_r = prompt_resumer_cctp + text_from_file
    result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_r}])
    #result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_r}])
    #pprint(result, compact=True)
    #bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s \n" + result.message.content)
    bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s")
    
    #JSON
    prompt_full_j = prompt_json_cctp + result.message.content
    result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_j}], format='json')
    #pprint(result, compact=True)
    bid_utils.print_text_wrapped(f"\nLLM response pour JSON: in {result['total_duration']/10**9:.0f}s \n{result.message.content}")
    return(result.message.content)


def run_model_reglement(text_from_file, model_name):
    #Init system:
    llmclient.chat(model=model_name, options=model_options, messages=[{'role':'system','content': prompt_role}])
    
    #Resumer
    prompt_full_r = prompt_resumer_regl + text_from_file
    result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_r}])
    #result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_r}])
    #pprint(result, compact=True)
    #bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s \n" + result.message.content)
    bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s")

    #JSON
    prompt_full_j = prompt_json_regl + result.message.content
    result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_j}], format='json')
    #pprint(result, compact=True)
    bid_utils.print_text_wrapped(f"\nLLM response pour JSON: in {result['total_duration']/10**9:.0f}s \n{result.message.content}")
    return(result.message.content)


def run_model_aapc(text_from_file, model_name):
    #Init system:
    llmclient.chat(model=model_name, options=model_options, messages=[{'role':'system','content': prompt_role}])
    
    #Resumer
    prompt_full_r = prompt_resumer_aapc + text_from_file
    result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_r}])
    #result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_r}])
    #pprint(result, compact=True)
    #bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s \n" + result.message.content)
    bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s")

    #JSON
    prompt_full_j = prompt_json_aapc + result.message.content
    result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_j}], format='json')
    #pprint(result, compact=True)
    bid_utils.print_text_wrapped(f"\nLLM response pour JSON: in {result['total_duration']/10**9:.0f}s \n{result.message.content}")
    return(result.message.content)


def run_model_ccap(text_from_file, model_name):
    #Init system:
    llmclient.chat(model=model_name, options=model_options, messages=[{'role':'system','content': prompt_role}])
    
    #Resumer
    prompt_full_r = prompt_resumer_ccap + text_from_file
    result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_r}])
    #result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_r}])
    #pprint(result, compact=True)
    #bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s \n" + result.message.content)
    bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s")
    
    #JSON
    prompt_full_j = prompt_json_ccap + result.message.content
    result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_j}], format='json')
    #pprint(result, compact=True)
    bid_utils.print_text_wrapped(f"\nLLM response pour JSON: in {result['total_duration']/10**9:.0f}s \n{result.message.content}")
    return(result.message.content)


def run_list_model_chat(text_from_file):
    for i in range(len(list_model)):
        model_name = list_model[i]
        print("\n==========================================")
        print(f"===== Test model :{i+1}/{len(list_model)}: {model_name} =========\n")
        
        #Init system:
        llmclient.chat(model=model_name, options=model_options, messages=[{'role':'system','content': prompt_role}])
        
        #Resumer
        prompt_full_r = prompt_task_resumer2 + text_from_file
        result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_r}])
        #result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_r}])
        #pprint(result, compact=True)
        bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s \n" + result.message.content)

        #JSON
        #prompt_full_j = prompt_task_json_uniq3 + result.message.content
        prompt_full_j = prompt_task_json_uniq3 + result.message.content
        result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_j}], format='json')
        #pprint(result, compact=True)
        bid_utils.print_text_wrapped(f"\nLLM response pour JSON: in {result['total_duration']/10**9:.0f}s \n{result.message.content}")


def run_list_model_chat_1proj_multiple_file():

    condition_mask = (df_consult_ebp['AO_docs'] == True) & \
                        (df_consult_ebp['AO_doc_type'] == "CCTP") 

    matching_ebp_ids = df_consult_ebp.loc[condition_mask, 'ID EBP'].unique()
    print(f"il y a {len(matching_ebp_ids)} projets avec au moins 1 document du type CCTP")
    
    if len(matching_ebp_ids) > 0:
        # Determine how many unique IDs we can actually select (min of 5 or available count)
        num_to_select = min(1, len(matching_ebp_ids))
        selected_ebp_ids = np.random.choice(matching_ebp_ids, num_to_select, replace=False)
        for ebp_id in selected_ebp_ids:

            filtered_row = df_consult_ebp[df_consult_ebp['ID EBP'] == ebp_id].iloc[0]
            print(f"ID:{ebp_id} statut_consult: {filtered_row['statut_consult']}")
            text_multi_files = ""
            list_file_consult = df_consult_ebp[(df_consult_ebp['ID EBP'] == ebp_id) & (df_consult_ebp['AO_doc_type'] == 'CCTP')]['file_path']
            print(f"Il y a {len(list_file_consult)} fichiers dans la dossier AO consultation pour ce projet {ebp_id}")
            print(f"Ville:{filtered_row['Ville']}  Entreprise:{filtered_row['Entreprise']}  Mission:{filtered_row['Mission']}")
            for file_path in list_file_consult:
                #nb_segment = file_path.split("\\")
                #print("==> ", nb_segment[-1].strip())
                bid_utils.path_to_link(file_path.strip(), option=None)
                text_file = bid_utils.loadpdf_as_text(file_path.strip())
                text_multi_files += text_file + "\n" # Contatenate text
                
            if text_multi_files != "":
                print("Run models pour les fichier(s) selectionné(s), lg text input:", len(text_multi_files))
                run_list_model_chat(text_multi_files)


In [15]:
## Liste all files for given consultation type CCTP (EBP IP)
## Run le modele pour resumer et json

# Preferable de faire une extraction par type de fichier (CCTP, CCAP, ..) avec un prompt dédié.
# donc regroupement par type de fichier
"""
def run_1model_nCCxP_proj(model_name="gemma2:9b", nb_of_projet_to_test = 1):
    #model_name = "gemma2:9b"
    #nb_of_projet_to_test = 5
    count_id = 1

    # liste des ID correspondants au mask
    condition_mask = (df_consult_ebp['AO_docs'] == True) & \
                        (df_consult_ebp['AO_doc_type'] == "CCxP") 

    matching_ebp_ids = df_consult_ebp.loc[condition_mask, 'ID EBP'].unique()
    print(f"il y a {len(matching_ebp_ids)} projets avec au moins 1 document du type CCxP")
    print(f"on teste l'extraction avec {nb_of_projet_to_test} projets, par le modele {model_name}\n")

    #Init system:
    result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'system','content': prompt_role}])

    if len(matching_ebp_ids) > 0:
        # Determine how many unique IDs we can actually select (min of 5 or available count)
        num_to_select = min(nb_of_projet_to_test, len(matching_ebp_ids))

        # Randomly select the desired number of unique EBP IDs
        selected_ebp_ids = np.random.choice(matching_ebp_ids, num_to_select, replace=False)

        #print(f"\n{num_to_select} ID(s) EBP aléatoire(s) sélectionné(s) correspondant au filtre:")
        for ebp_id in selected_ebp_ids:

            filtered_row = df_consult_ebp[df_consult_ebp['ID EBP'] == ebp_id].iloc[0]
            print(f"\n{count_id}/{nb_of_projet_to_test} => ID:{ebp_id} statut_consult: {filtered_row['statut_consult']}")

            text_multi_files = ""
            list_file_consult = df_consult_ebp[(df_consult_ebp['ID EBP'] == ebp_id) & (df_consult_ebp['AO_doc_type'] == 'CCxP')]['file_path']
            print(f"Il y a {len(list_file_consult)} fichiers dans la dossier AO consultation pour ce projet {ebp_id}")
            print(f"Ville:{filtered_row['Ville']}  Entreprise:{filtered_row['Entreprise']}  Mission:{filtered_row['Mission']}")
            for file_path in list_file_consult:
                #nb_segment = file_path.split("\\")
                #print("==> ", nb_segment[-1].strip())
                bid_utils.path_to_link(file_path.strip(), option=None)
                text_file = bid_utils.loadpdf_as_text(file_path.strip())
                text_multi_files += text_file + "\n" # Contatenate text
                
            if text_multi_files != "":
                print("Run model pour les fichier(s) selectionné(s), lg text input:", len(text_multi_files))

                #Resumer
                prompt_full = prompt_task_resumer + text_multi_files
                result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full}])
                bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s \n")

                #JSON
                prompt_full = prompt_task_json + text_multi_files
                result = llmclient.chat(model=model_name, options=model_options_json, messages=[{'role':'user','content':prompt_full}])
                bid_utils.print_text_wrapped(f"\nLLM response pour JSON: in {result['total_duration']/10**9:.0f}s \n{result.message.content}")

                cctp_json = bid_utils.print_json_info_cctp(result.message.content)
                
                # extrait les infos du JSON
                if cctp_json != "":
                    parsed_json = {} # Initialize an empty dictionary
                    try:
                        parsed_json = json.loads(cctp_json)
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON string: {e}")
                    except TypeError:
                        print("Input is not a string.") 
                    
                    # Extrait chaque clef
                    lieu_value = parsed_json.get("Lieu du Chantier")
                    type_travaux_value = parsed_json.get("Type de Travaux")
                    planning_value = parsed_json.get("Planning previsionnel")
                    duree_travaux_value = parsed_json.get("Durée Prévisionnelle des Travaux (en mois)")
                    prix_travaux_value = parsed_json.get("Prix des travaux (en euros)")
                    moa_value = parsed_json.get("Maitre ouvrage")
                    moe_value = parsed_json.get("Maitre oeuvre")

                    # Mise à jour du df_consult_elevated
                    mask = df_consult_elevated['ID EBP'] == ebp_id
                    if lieu_value is not None:
                        df_consult_elevated.loc[mask, 'lieu'] = lieu_value
                    if type_travaux_value is not None:
                        df_consult_elevated.loc[mask, 'type travaux'] = type_travaux_value
                    if duree_travaux_value is not None:
                        df_consult_elevated.loc[mask, 'duree travaux'] = duree_travaux_value
                    if planning_value is not None:
                        df_consult_elevated.loc[mask, 'planning'] = planning_value
                    if prix_travaux_value is not None:
                        df_consult_elevated.loc[mask, 'prix travaux'] = prix_travaux_value
                    if moa_value is not None:
                        df_consult_elevated.loc[mask, 'maitre ouvrage'] = moa_value
                    if moe_value is not None:
                        df_consult_elevated.loc[mask, 'maitre oeuvre'] = moe_value
            else:
                print("Pas de fichiers consult à processer")
            count_id += 1
    else:
        print("\nAucune ligne trouvée avec 'statut_consult' == 'Chantiers'")

"""

def run_1model_gen_nCCxP_proj(model_name="gemma2:9b", nb_of_projet_to_test = 1):
    #model_name = "gemma2:9b"
    #nb_of_projet_to_test = 5
    count_id = 1

    # liste des ID correspondants au mask
    condition_mask = (df_consult_ebp['AO_docs'] == True) & \
                        (df_consult_ebp['AO_doc_type'] == "CCTP") 

    matching_ebp_ids = df_consult_ebp.loc[condition_mask, 'ID EBP'].unique()
    print(f"il y a {len(matching_ebp_ids)} projets avec au moins 1 document du type CCTP")
    print(f"on teste l'extraction avec {nb_of_projet_to_test} projets, par le modele {model_name}\n")

    if len(matching_ebp_ids) > 0:
        # Determine how many unique IDs we can actually select (min of 5 or available count)
        num_to_select = min(nb_of_projet_to_test, len(matching_ebp_ids))

        # Randomly select the desired number of unique EBP IDs
        selected_ebp_ids = np.random.choice(matching_ebp_ids, num_to_select, replace=False)

        #print(f"\n{num_to_select} ID(s) EBP aléatoire(s) sélectionné(s) correspondant au filtre:")
        for ebp_id in selected_ebp_ids:

            filtered_row = df_consult_ebp[df_consult_ebp['ID EBP'] == ebp_id].iloc[0]
            print(f"\n{count_id}/{nb_of_projet_to_test} => ID:{ebp_id} statut_consult: {filtered_row['statut_consult']}")

            text_multi_files = ""
            list_file_consult = df_consult_ebp[(df_consult_ebp['ID EBP'] == ebp_id) & (df_consult_ebp['AO_doc_type'] == 'CCxP')]['file_path']
            print(f"Il y a {len(list_file_consult)} fichiers dans la dossier AO consultation pour ce projet {ebp_id}")
            print(f"Ville:{filtered_row['Ville']}  Entreprise:{filtered_row['Entreprise']}  Mission:{filtered_row['Mission']}")
            for file_path in list_file_consult:
                #nb_segment = file_path.split("\\")
                #print("==> ", nb_segment[-1].strip())
                bid_utils.path_to_link(file_path.strip(), option=None)
                text_file = bid_utils.loadpdf_as_text(file_path.strip())
                text_multi_files += text_file + "\n" # Contatenate text
            

            if text_multi_files != "":
                # supprime les espaces vides
                text_input = re.sub(r'\s+', ' ', text_multi_files).strip()
                print("Run model pour les fichier(s) selectionné(s), lg text input:", len(text_multi_files), len(text_input))

                #Init system:
                llmclient.chat(model=model_name, options=model_options, messages=[{'role':'system','content': prompt_role}])
                
                #Resumer
                prompt_full_r = prompt_task_resumer2 + text_input
                result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_r}])
                #pprint(result, compact=True)
                bid_utils.print_text_wrapped(f"\nLLM response pour resumer: in {result['total_duration']/10**9:.0f}s \n" + result.message.content)

                #JSON
                prompt_full_j = prompt_task_json_uniq3 + result.message.content
                result = llmclient.chat(model=model_name, options=model_options, messages=[{'role':'user','content':prompt_full_j}], format='json')
                #pprint(result, compact=True)
                bid_utils.print_text_wrapped(f"\nLLM response pour JSON: in {result['total_duration']/10**9:.0f}s \n{result.message.content}")

                #cctp_json = bid_utils.print_json_info_cctp(result.message.content)
                parsed_json = json.loads(result.message.content)

                # extrait les infos du JSON
                if parsed_json != "":
                    parsed_json = {} # Initialize an empty dictionary
                    try:
                        parsed_json = json.loads(result.message.content)
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON string: {e}")
                    except TypeError:
                        print("Input is not a string.") 
                    
                    # Extrait chaque clef
                    lieu_value = parsed_json.get("Lieu du Chantier")
                    type_travaux_value = parsed_json.get("Type de Travaux")
                    planning_concept_value = parsed_json.get("Planning phase conception")
                    planning_real_value = parsed_json.get("Planning phase realistion")
                    duree_travaux_value = parsed_json.get("Duree des travaux")
                    prix_travaux_value = parsed_json.get("Prix des travaux")
                    cat_sps_value = parsed_json.get("Categorie operation SPS")
                    moa_value = parsed_json.get("Maitre ouvrage")
                    moe_value = parsed_json.get("Maitre oeuvre")

                    # Mise à jour du df_consult_elevated
                    mask = df_consult_elevated['ID EBP'] == ebp_id
                    if lieu_value is not None:
                        df_consult_elevated.loc[mask, 'lieu'] = lieu_value
                    if type_travaux_value is not None:
                        df_consult_elevated.loc[mask, 'type travaux'] = type_travaux_value
                    if duree_travaux_value is not None:
                        df_consult_elevated.loc[mask, 'duree travaux'] = duree_travaux_value
                    if planning_concept_value is not None:
                        df_consult_elevated.loc[mask, 'planning conception'] = planning_concept_value
                    if planning_real_value is not None:
                        df_consult_elevated.loc[mask, 'planning realisation '] = planning_real_value
                    if prix_travaux_value is not None:
                        df_consult_elevated.loc[mask, 'prix travaux'] = prix_travaux_value
                    if moa_value is not None:
                        df_consult_elevated.loc[mask, 'maitre ouvrage'] = moa_value
                    if moe_value is not None:
                        df_consult_elevated.loc[mask, 'maitre oeuvre'] = moe_value
                    if cat_sps_value is not None:
                        df_consult_elevated.loc[mask, 'Categorie operation SPS'] = cat_sps_value
            else:
                print("Pas de fichiers consult à processer")
            count_id += 1
    else:
        print("\nAucune ligne trouvée avec 'statut_consult' == 'Chantiers'")



In [16]:
## Teste les modeles 

#list_model = ["gemma3:12b", "gemma2:9b", "llama3.1:8b", "mistral:7b"]
# gemma3:12b : very long , not performing so well.
# mistral:7b : does not respect instructions (info request, JSON structure)
# "qwen3:8b"
#list_model = ["gemma3:4b", "llama3.1:8b", "qwen3:8b"]
list_model = ["llama3.2:latest"] # + rapides, reponses longues en Anglais, courtes/json en Francais.

# Select 1 file for the test
condition_mask = (df_consult_ebp['AO_docs'] == True) & \
                    (df_consult_ebp['AO_doc_type'] == "CCTP") 

matching_ebp_ids = df_consult_ebp.loc[condition_mask, 'ID EBP'].unique()
print(f"il y a {len(matching_ebp_ids)} projets avec au moins 1 document du type CCTP")
if len(matching_ebp_ids) > 0:
    # Determine how many unique IDs we can actually select (min of 5 or available count)
    #num_to_select = min(1, len(matching_ebp_ids))
    selected_ebp_id = np.random.choice(matching_ebp_ids, 1, replace=False)
    selected_ebp_id = ["no EBP 9717"] # impose ce projet
    
    filtered_row = df_consult_ebp[df_consult_ebp['ID EBP'] == selected_ebp_id[0]].iloc[0]
    print(f"\n ***** ID:{selected_ebp_id} statut_consult: {filtered_row['statut_consult']} ****** ")
    print(f"Ville:{filtered_row['Ville']}  Entreprise:{filtered_row['Entreprise']}  Mission:{filtered_row['Mission']}")
    
    text_multi_files = ""
    file_consult = df_consult_ebp[(df_consult_ebp['ID EBP'] == selected_ebp_id[0]) & (df_consult_ebp['AO_doc_type'] == 'CCTP')]['file_path'].iloc[0]
    #print("File ==> ", file_consult)
    bid_utils.path_to_link(file_consult.strip(), option=None)
    text_to_analyse = bid_utils.loadpdf_as_text(file_consult.strip())

# Compare model results for same file same prompt , models to test is defined in list_model
#run_list_model_chat(text_to_analyse)   #best model so far: llama3.1:8b

#or Compare model results for same projet (with multiple files), same prompt , models to test is defined in list_model
#run_list_model_chat_1proj_multiple_file(text_to_analyse)



il y a 225 projets avec au moins 1 document du type CCTP

 ***** ID:['no EBP 9717'] statut_consult: Perdu ****** 
Ville:ST BARTHELEMY D'ANJOU  Entreprise:MAIRIE  Mission:Espaces publics et rénovation bâtiments


In [17]:
#run_1model_nCCxP_proj(model_name="llama3.1:8b", nb_of_projet_to_test = 2)
#run_1model_gen_nCCxP_proj(model_name="llama3.2:latest", nb_of_projet_to_test = 10)

In [ ]:
# Teste plusieurs models sur l'enchaienement d'analyse de fichiers pour 1 EBP ID
def test_models_for_1_ebp(list_model_to_test):
    ebp_id = ["no EBP 13"] # impose ce projet "no EBP 13" => CCTP,CCAP, Regl  / #no EBP 9717
    list_consult_type = ['Lettre Consult', 'CCP', 'CCTP', 'CCAP', 'Planning', 'AAPC', 'Reglement']

    #filtered_row = df_consult_ebp[df_consult_ebp['ID EBP'] == selected_ebp_id[0]].iloc[0]
    #print(f"\n ***** ID:{selected_ebp_id} statut_consult: {filtered_row['statut_consult']} ****** ")
    filtered_row = df_consult_ebp[df_consult_ebp['ID EBP'] == ebp_id].iloc[0]
    print(f"\n ***** ID:{ebp_id} statut_consult: {filtered_row['statut_consult']} ***** ")
    print(f"Ville:{filtered_row['Ville']}  Entreprise:{filtered_row['Entreprise']}  Mission:{filtered_row['Mission']}")
    
    for model_name in list_model_to_test:
        print(f" ===================== {model_name} ==========================")
        for doc_type in list_consult_type:
            list_file_consult = df_consult_ebp[(df_consult_ebp['ID EBP'] == ebp_id) & (df_consult_ebp['AO_doc_type'] == doc_type)]['file_path']
        
            if not list_file_consult.empty:
                print("Doc type:",doc_type)
                text_multi_files = ""
                for file_path in list_file_consult:
                    #nb_segment = file_path.split("\\")
                    #print("==> ", nb_segment[-1].strip())
                    bid_utils.path_to_link(file_path.strip(), option=None)
                    text_file = bid_utils.loadpdf_as_text(file_path.strip())
                    text_multi_files += text_file + "\n" # Contatenate text
                
                # Trim text to suppress long' '
                text_input = re.sub(r'\s+', ' ', text_multi_files).strip()
                
                # Run model for each doc type
                if doc_type == "CCTP" or doc_type == "CCP" or doc_type == "Lettre Consult" or doc_type == "Planning" :
                    json_result = run_model_CCTP(text_input, model_name)
                    bid_utils.update_df_with_json_cctp(json_result , ebp_id, df_consult_elevated)
                    nb_model_called += 1
                if doc_type == "Reglement" or doc_type == "Lettre Consult":
                    json_result = run_model_reglement(text_input, model_name)
                    bid_utils.update_df_with_json_regl(json_result , ebp_id, df_consult_elevated)
                    nb_model_called += 1
                if doc_type == "AAPC" or doc_type == "Planning":
                    json_result = run_model_aapc(text_input, model_name)
                    bid_utils.update_df_with_json_aapc(json_result , ebp_id, df_consult_elevated)
                    nb_model_called += 1
                if doc_type == "CCAP" or doc_type == "CCP" or doc_type == "Planning":
                    json_result = run_model_ccap(text_input, model_name)
                    bid_utils.update_df_with_json_ccap(json_result , ebp_id, df_consult_elevated)
                    nb_model_called += 1

#list_model = ["gemma3:4b", "gemma3:12b", "gemma2:9b", "llama3.1:8b", "llama3.2:latest", "qwen3:8b"]
list_model = ["gemma3:4b",  "gemma2:9b", "llama3.1:8b", "llama3.2:latest"]

#test_models_for_1_ebp(list_model)

In [ ]:
#### Test enchainement des docs d'un projet pour cumuler les infos.

# modele fixé
model_name = "gemma3:4b" # ou "llama3.2:latest" "gemma3:4b"

# Liste ordonnées des types de doc Consult à explorer
list_consult_type = ['Lettre Consult', 'CCP', 'CCTP', 'CCAP', 'Planning', 'AAPC', 'Reglement']

consult_mask = (df_consult_ebp['AO_docs'] == True) 
#cctp_mask = df_consult_ebp['AO_doc_type'] == "CCTP"

consult_ebp_ids = df_consult_ebp.loc[consult_mask, 'ID EBP'].unique()
print(f"il y a {len(consult_ebp_ids)} projets avec au moins 1 document du type 'Consultation' ")
print("nb de fichiers AO_docs avec un type identifié:",len(df_consult_ebp['AO_doc_type']) - len(df_consult_ebp[df_consult_ebp['AO_doc_type']=='no type']))

count = 0
start_exec = 0
max_count = 5
nb_model_called = 0
count_ebp_model = 0 

for ebp_id in consult_ebp_ids:

    filtered_row = df_consult_ebp[df_consult_ebp['ID EBP'] == ebp_id].iloc[0]
    print(f"\n ***** {count+1}/{max_count}:  ID:{ebp_id} statut_consult: {filtered_row['statut_consult']} ***** ")
    print(f"Ville:{filtered_row['Ville']}  Entreprise:{filtered_row['Entreprise']}  Mission:{filtered_row['Mission']}")

    for doc_type in list_consult_type:
        list_file_consult = df_consult_ebp[(df_consult_ebp['ID EBP'] == ebp_id) & (df_consult_ebp['AO_doc_type'] == doc_type)]['file_path']
        flag_callmodel = False
        if not list_file_consult.empty:
            print("Doc type:",doc_type)
            text_multi_files = ""
            for file_path in list_file_consult:
                #nb_segment = file_path.split("\\")
                #print("==> ", nb_segment[-1].strip())
                bid_utils.path_to_link(file_path.strip(), option=None)
                text_file = bid_utils.loadpdf_as_text(file_path.strip())
                text_multi_files += text_file + "\n" # Contatenate text
            
            # Trim text to suppress long' '
            text_input = re.sub(r'\s+', ' ', text_multi_files).strip()

            if count >= start_exec:
                # Run model for each doc type
                flag_callmodel = True
                if doc_type == "CCTP" or doc_type == "CCP" or doc_type == "Lettre Consult" or doc_type == "Planning" :
                    json_result = run_model_CCTP(text_input, model_name)
                    bid_utils.update_df_with_json_cctp(json_result , ebp_id, df_consult_elevated)
                    nb_model_called += 1
                if doc_type == "Reglement" or doc_type == "Lettre Consult":
                    json_result = run_model_reglement(text_input, model_name)
                    bid_utils.update_df_with_json_regl(json_result , ebp_id, df_consult_elevated)
                    nb_model_called += 1
                if doc_type == "AAPC" or doc_type == "Planning":
                    json_result = run_model_aapc(text_input, model_name)
                    bid_utils.update_df_with_json_aapc(json_result , ebp_id, df_consult_elevated)
                    nb_model_called += 1
                if doc_type == "CCAP" or doc_type == "CCP" or doc_type == "Planning":
                    json_result = run_model_ccap(text_input, model_name)
                    bid_utils.update_df_with_json_ccap(json_result , ebp_id, df_consult_elevated)
                    nb_model_called += 1
    count_ebp_model += 1
    if count > max_count:
        break
    count += 1

il y a 926 projets avec au moins 1 document du type 'Consultation' 
nb de fichiers AO_docs avec un type identifié: 830

 ***** 1/5:  ID:no EBP 1 statut_consult: Perdu ***** 
Ville:BOUGUENAIS  Entreprise:BGTA Ministère environnement  Mission:Sécurisation BGTA
Doc type: Reglement



LLM response pour resumer: in 123s

LLM response pour JSON: in 12s
{
  "Critere Prix": 60,
  "Critere Technique": {
    "Moyen Humain et Experience": 0,
    "Methodologie": 0,
    "Cohérence du temps": 0,
    "Comprehension des enjeux": 0
  },
  "Prix des travaux": 33920,
  "Duree des travaux": 7
}

 ***** 2/5:  ID:no EBP 3 statut_consult: Devis ***** 
Ville:BOUGUENAIS  Entreprise:MAIRIE  Mission:Salle de sport Joel Dubois et Cossec
Doc type: CCTP



LLM response pour resumer: in 181s

LLM response pour JSON: in 21s
{
  "Nom Chantier": "Installation Électrique APD",
  "Lieu du Chantier": "Complexe APD",
  "Maitre ouvrage": "Non spécifié",
  "Maitre oeuvre": "Non spécifié",
  "Type de Travaux": "Installation électrique",
  "Planning phase conception": "Non spécifié",
  "Planning phase realisation": "Non spécifié",
  "Prix des travaux": 0,
  "Duree des travaux": 0,
  "Categorie operation SPS": "Non spécifié"
}

 ***** 3/5:  ID:no EBP 13 statut_consult: Perdu ***** 
Ville:ST NAZAIRE  Entreprise:SONADEV  Mission:CONSTRUCTION GROUPE SCOLAIRE
Doc type: CCTP



LLM response pour resumer: in 136s

LLM response pour JSON: in 25s
{
  "Nom Chantier": "Groupe scolaire Jean Zay – Saint Nazaire",
  "Lieu du Chantier": "Ville: Saint Nazaire\nCommune: Saint Nazaire\nDépartement: Non
spécifié\nRue: Non spécifié",
  "Maitre ouvrage": "SONADEV TERRITOIRES PUBLICS",
  "Maitre oeuvre": "Non spécifié",
  "Type de Travaux": "Construction d’un groupe scolaire, d’un multi accueil et d’un
plateau sportif.",
  "Planning phase conception": "Date: 12-juin-25\nDurée Prévisionnelle: 18,00 (jours)",
  "Planning phase realisation": "Date: 5-sept.-25\nDurée Prévisionnelle: 85,00 (jours)",
  "Prix des travaux": "Non spécifié",
  "Duree des travaux": "85,00 (jours) – phase réalisation",
  "Categorie operation SPS": "II (Catégorie 2 ou 3)"
}
Doc type: AAPC


c:\DocPerso\Dev_Python\ProjetsPerso\BidManagement\bid_utils.py:199: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '85,00 (jours) – phase réalisation' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_update.loc[mask, 'cctp duree travaux'] = duree_travaux_value



LLM response pour resumer: in 23s

LLM response pour JSON: in 11s
{
  "Mission": "Construction d’un groupe scolaire sur le quartier KERLEDE.",
  "Lieu du Chantier": "Ville: Saint-Nazaire\nCommune: Saint-Nazaire\nDépartement: 44\nRue:
rue Ferdinand Buisson",
  "Maitre ouvrage": "SONADEV TERRITOIRES PUBLICS",
  "Lot": null,
  "Tranche": "oui",
  "Prix des travaux": 8600000,
  "Duree des travaux": 67
}
Doc type: Reglement



LLM response pour resumer: in 196s


In [ ]:
print("nb of count_ebp_model",count_ebp_model)
print("nb of nb_model_called",nb_model_called, "\n")

#f_consult_ebp.loc[consult_mask].head()
df_consult_elevated.loc[consult_mask].info()

In [ ]:
df_consult_elevated.columns

In [ ]:
# show key info per columns
keywords = ['cctp', 'aapc', 'regl', 'ccp']
list_col_added = [col for col in df_consult_elevated.columns if any(keyword in col for keyword in keywords)]
mask_non_null = df_consult_elevated[list_col_added].map(lambda x: pd.notna(x) and (x != 0 or x != ""))
count_non_null_rows = mask_non_null.any(axis=1).sum()

filtered_df = df_consult_elevated[mask_non_null.any(axis=1)]
TableReport(filtered_df)